# bs

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd


In [2]:


head = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'}
data = []
with requests.Session() as s:
    for page in range(1,3):
        url = f"https://www.saramin.co.kr/zf_user/jobs/list/job-category?cat_mcls=2&page={page}&page_count=1000&sort=RD"
        r = s.get(url, headers=head)
        data.extend(r.json())
        if r.status_code != 200:
            break


ConnectTimeout: HTTPSConnectionPool(host='www.saramin.co.kr', port=443): Max retries exceeded with url: /zf_user/jobs/list/job-category?cat_mcls=2&page=1&page_count=1000&sort=RD (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002686EFB0F90>, 'Connection to www.saramin.co.kr timed out. (connect timeout=None)'))

In [3]:
data

[]

In [ ]:
    print(r, page)
    soup = BeautifulSoup(r.text)
    box_item_list = soup.select('div.box_item')


    for box_item in box_item_list:
        temp = {}
        temp['company_name'] = box_item.select_one('.company_nm > .str_tit').text.strip()
        temp['company_sn'] = box_item.select_one('.interested_corp')['csn']
        temp['is_headhunting'] = '헤드헌팅' in box_item.select_one(".company_nm").text
        temp['job_idx'] = box_item.find('button', attrs={'rec_idx': True})['rec_idx']
        temp['job_title'] = box_item.select_one('.notification_info a.str_tit')['title']
        temp['job_link'] = box_item.select_one('.notification_info a.str_tit')['href']
        temp['job_sectors'] = [i.text for i in box_item.select('.notification_info .job_meta .job_sector span')]
        work_place_element = box_item.select_one('.recruit_info .work_place')
        temp['work_place'] = work_place_element.text if work_place_element is not None else None
        temp['career_info'] = box_item.select_one('.recruit_info .career').text
        temp['education_info'] = box_item.select_one('.recruit_info .education').text
        temp['support_date'] = box_item.select_one('.support_info .date').text
        temp['deadline'] = box_item.select_one('.support_info .deadlines').text
        data.append(temp)

df = pd.DataFrame(data)
df['update'] = pd.to_datetime('now')
df.to_csv('saram.csv', index=False)

In [29]:
df  = pd.read_csv('saram.csv')
df['update'] = pd.to_datetime('now')


In [30]:
def process_deadline(row):
    x = row['deadline']

    word = x.split(' ')[0]

    if '일' in word:
        delta = pd.Timedelta(days=int(word.split('일')[0]))
    elif '시간' in word:
        delta = pd.Timedelta(hours=int(word.split('시간')[0]))
    elif '분' in word:
        delta = pd.Timedelta(minutes=int(word.split('분')[0]))

    if '등록' in x:
        row['resistered'] = row['update'] - delta
    elif '수정' in x:
        row['modified'] = row['update'] - delta
    else:
        pass
    
    return row

df.apply(process_deadline, axis=1)


,career_info,company_name,company_sn,deadline,education_info,is_headhunting,job_idx,job_link,job_sectors,job_title,modified,resistered,support_date,update,work_place
0,6 ~ 12년 · 정규직,드림픽셀,b2lOcDJyQmhWNHdpRHJ3eVlLMnRoUT09,7분 전 등록,"대학(2,3년)↑",True,47544195,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['게임개발', '백엔드/서버개발', '앱개발', '웹개발', '웹마스터']","(중소기업-서울) 웹,앱 개발자 (풀스택 개발자) 모집-헤드헌팅",NaT,2024-02-08 17:47:45.942361,~03.09(토),2024-02-08 17:54:45.942361,서울 성동구
1,3 ~ 10년 · 정규직,코리아휴먼리스오스시스템(주),bU5SNDIweDRqdjdpVFpWMlpRUVJ6dz09,10분 전 등록,대학교(4년)↑,True,47544168,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['머신러닝', '알고리즘', '영상처리', '이미지프로세싱', '컴퓨터비전']",[코스닥/금천구] 머신비전장비 AI기반 SW개발(대리~과장),NaT,2024-02-08 17:44:45.942361,~02.16(금),2024-02-08 17:54:45.942361,서울 금천구
2,경력무관 · 정규직 외,제이치에이치컴퍼니,V0d5SkZrVEFBNnkvSkJ2bnBRVmlJZz09,11분 전 등록,학력무관,False,47544154,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['웹마스터', '유지보수', '퍼블리셔', '검색엔진', '광고마케팅']",웹사이트 관리 / 서비스지원 사무직 채용/포토샵일러스트 능숙자,NaT,2024-02-08 17:43:45.942361,~03.09(토),2024-02-08 17:54:45.942361,서울 강남구
3,경력 5년↑ · 정규직,(주)잇올,V2tiKzhmVGJBREU3ZU5xR1JmRTZwQT09,20분 전 등록,"대학(2,3년)↑",False,47544065,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['데이터분석가', '백엔드/서버개발', '앱개발', '웹개발', '빅데이터']",[잇올그룹] 웹/앱 기획자 채용,NaT,2024-02-08 17:34:45.942361,~03.09(토),2024-02-08 17:54:45.942361,서울 서초구 외
4,경력무관 · 인턴직,(주)에임블,RDZFeHVuREN6Vkdad0pRVDNmdllIUT09,22분 전 등록,대학교(4년)↑,False,47544036,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['데이터분석가', 'Linux', 'Python', 'R', 'Tensorflow']",AI 신약 개발 R&amp;D 연구원 채용 (정규직 전환 인턴),NaT,2024-02-08 17:32:45.942361,~03.09(토),2024-02-08 17:54:45.942361,서울 강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,경력 1년↑ · 정규직,(주)이에스이,cStudWlYajZBZDRvNzJuTGxnODhTUT09,3시간 전 수정,"대학(2,3년)↑",False,47518552,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['기술지원', 'H/W', 'S/W', '연구원', '기술연구']",이에스이) 기술개발 사원 채용 공고,2024-02-08 14:54:45.942361,NaT,~03.06(수),2024-02-08 17:54:45.942361,인천 미추홀구
1996,경력무관 · 계약직 외,(주)맨파워그룹코리아,SmJ4OVQyczYwSExPNWdMZlh4SGtydz09,3일 전 등록,"대학(2,3년)↑",False,47518547,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['기술지원', '유지보수', '정보보안', 'IT컨설팅', 'QA/테스터']",[서울역] 주방용품 제조 중견기업 IT 팀원 채용 (12M/신입가능),NaT,2024-02-05 17:54:45.942361,채용시,2024-02-08 17:54:45.942361,서울 중구
1997,신입 · 인턴직,네이버랩스(주),d2lxdmpQWXVNMCt6bmI2eEZ2NWVNQT09,3일 전 등록,석사↑,False,47518510,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['C++', 'Python', '3D디자인']",[NAVER] 네이버랩스 3D Vision Research Engineer (인턴십),NaT,2024-02-05 17:54:45.942361,~02.19(월),2024-02-08 17:54:45.942361,경기 성남시
1998,경력 3년↑ · 정규직 외,주식회사 더씨에스알,ZUlaTFEzYlVaWGw2TEF4dWNIRWhxQT09,3일 전 등록,대학교(4년)↑,False,47518488,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['Java', '유지보수', '빅데이터', '클라우드']",인하우스 웹서비스 백엔드 개발자 채용,NaT,2024-02-05 17:54:45.942361,~03.06(수),2024-02-08 17:54:45.942361,서울 종로구 외


In [22]:
df

,company_name,company_sn,is_headhunting,job_idx,job_title,job_link,job_sectors,work_place,career_info,education_info,support_date,deadline,update
0,드림픽셀,b2lOcDJyQmhWNHdpRHJ3eVlLMnRoUT09,True,47544195,"(중소기업-서울) 웹,앱 개발자 (풀스택 개발자) 모집-헤드헌팅",/zf_user/jobs/relay/view?view_type=list&rec_id...,"['게임개발', '백엔드/서버개발', '앱개발', '웹개발', '웹마스터']",서울 성동구,6 ~ 12년 · 정규직,"대학(2,3년)↑",~03.09(토),7분 전 등록,2024-02-08 17:04:59.203657
1,코리아휴먼리스오스시스템(주),bU5SNDIweDRqdjdpVFpWMlpRUVJ6dz09,True,47544168,[코스닥/금천구] 머신비전장비 AI기반 SW개발(대리~과장),/zf_user/jobs/relay/view?view_type=list&rec_id...,"['머신러닝', '알고리즘', '영상처리', '이미지프로세싱', '컴퓨터비전']",서울 금천구,3 ~ 10년 · 정규직,대학교(4년)↑,~02.16(금),10분 전 등록,2024-02-08 17:04:59.203657
2,제이치에이치컴퍼니,V0d5SkZrVEFBNnkvSkJ2bnBRVmlJZz09,False,47544154,웹사이트 관리 / 서비스지원 사무직 채용/포토샵일러스트 능숙자,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['웹마스터', '유지보수', '퍼블리셔', '검색엔진', '광고마케팅']",서울 강남구,경력무관 · 정규직 외,학력무관,~03.09(토),11분 전 등록,2024-02-08 17:04:59.203657
3,(주)잇올,V2tiKzhmVGJBREU3ZU5xR1JmRTZwQT09,False,47544065,[잇올그룹] 웹/앱 기획자 채용,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['데이터분석가', '백엔드/서버개발', '앱개발', '웹개발', '빅데이터']",서울 서초구 외,경력 5년↑ · 정규직,"대학(2,3년)↑",~03.09(토),20분 전 등록,2024-02-08 17:04:59.203657
4,(주)에임블,RDZFeHVuREN6Vkdad0pRVDNmdllIUT09,False,47544036,AI 신약 개발 R&amp;D 연구원 채용 (정규직 전환 인턴),/zf_user/jobs/relay/view?view_type=list&rec_id...,"['데이터분석가', 'Linux', 'Python', 'R', 'Tensorflow']",서울 강남구,경력무관 · 인턴직,대학교(4년)↑,~03.09(토),22분 전 등록,2024-02-08 17:04:59.203657
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,(주)이에스이,cStudWlYajZBZDRvNzJuTGxnODhTUT09,False,47518552,이에스이) 기술개발 사원 채용 공고,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['기술지원', 'H/W', 'S/W', '연구원', '기술연구']",인천 미추홀구,경력 1년↑ · 정규직,"대학(2,3년)↑",~03.06(수),3시간 전 수정,2024-02-08 17:04:59.203657
1996,(주)맨파워그룹코리아,SmJ4OVQyczYwSExPNWdMZlh4SGtydz09,False,47518547,[서울역] 주방용품 제조 중견기업 IT 팀원 채용 (12M/신입가능),/zf_user/jobs/relay/view?view_type=list&rec_id...,"['기술지원', '유지보수', '정보보안', 'IT컨설팅', 'QA/테스터']",서울 중구,경력무관 · 계약직 외,"대학(2,3년)↑",채용시,3일 전 등록,2024-02-08 17:04:59.203657
1997,네이버랩스(주),d2lxdmpQWXVNMCt6bmI2eEZ2NWVNQT09,False,47518510,[NAVER] 네이버랩스 3D Vision Research Engineer (인턴십),/zf_user/jobs/relay/view?view_type=list&rec_id...,"['C++', 'Python', '3D디자인']",경기 성남시,신입 · 인턴직,석사↑,~02.19(월),3일 전 등록,2024-02-08 17:04:59.203657
1998,주식회사 더씨에스알,ZUlaTFEzYlVaWGw2TEF4dWNIRWhxQT09,False,47518488,인하우스 웹서비스 백엔드 개발자 채용,/zf_user/jobs/relay/view?view_type=list&rec_id...,"['Java', '유지보수', '빅데이터', '클라우드']",서울 종로구 외,경력 3년↑ · 정규직 외,대학교(4년)↑,~03.06(수),3일 전 등록,2024-02-08 17:04:59.203657


# base 불러오기

In [2]:
import pandas as pd

df = pd.read_csv('saram14000.csv')

# 사명 변경된 적 있는지

In [64]:
final_df = df[df.groupby('company_sn')['company_name'].transform('nunique') > 1].sort_values(by='job_idx')[['company_sn', 'company_name']].drop_duplicates()
final_df['name_count'] = final_df.groupby('company_sn').cumcount() + 1
final_df.pivot_table(columns='name_count', index='company_sn', values='company_name', aggfunc='first')


name_count,1,2
company_sn,,
K2hxMUQzdHJ1NTBERGovS2pxb2JQQT09,(주)씨엔알리서치,씨엔알리서치
L2ZKR25TU2UxUHZLYUJHK0J4cHFZUT09,(주)이글루코퍼레이션,㈜이글루코퍼레이션
MDJMQStlRGNIQU51dFlJNFdzRjIwdz09,㈜스마트비전,(주)스마트비전
MDhmcTYxNTVVVFlDbWx4QnlQQ3kzZz09,(주)미투온,주식회사 미투온
ME4zdk5IMlQxdlJZOHMwVmhReUNtdz09,(주)알티올,알티올(주)
MFlJb2hybDJiYXQ4WHNJeWpubUVwQT09,(주)자이온아이티에스,주식회사 자이온아이티에스
MkVmMEk3N2VUTWFUeGljZnNCZUY0QT09,(주)틱스잇,(주) 틱스잇
NEdFei9JVjRIWm1iSFBFcllVbE5HUT09,(주)씨에이이테크놀러지,주식회사씨에이이테크놀러지
NEtmNzFnU0hLdXBnN3B3UFhqdGdGUT09,선우ENG㈜,선우이엔지(주)


# 등록일

In [70]:
df.deadline.value_counts().to_csv('test.csv')

In [5]:
df.deadline.str.find('등록').value_counts()

deadline
 6    8086
 5    3752
 7    1350
-1     776
 8       7
Name: count, dtype: int64